##Imports and data names

In [88]:
import pandas as pd
import numpy as np
import os
import warnings

In [8]:
COUNTRY_NAMES = "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/raw/master/all/all.csv"

In [55]:
GDP = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel"
gdp_para = "gdp"

gdp_percapita = "https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.CD?downloadformat=excel"
gdp_percapita_para = "gdp_percapita"

pop_growth = "https://api.worldbank.org/v2/en/indicator/SP.POP.GROW?downloadformat=excel"
pop_growth_para = "pop_growth"

life_exp_female = "https://api.worldbank.org/v2/en/indicator/SP.DYN.LE00.FE.IN?downloadformat=excel"
life_exp_female_para = "life_exp_female"

life_exp_male = "https://api.worldbank.org/v2/en/indicator/SP.DYN.LE00.MA.IN?downloadformat=excel"
life_exp_male_para = "life_exp_male"

mortality_male = "https://api.worldbank.org/v2/en/indicator/SP.DYN.AMRT.MA?downloadformat=excel"
mortality_male_para = "mortality_male"

mortality_female = "https://api.worldbank.org/v2/en/indicator/SP.DYN.AMRT.FE?downloadformat=excel"
mortality_female_para = "mortality_female"

death_rate = "https://api.worldbank.org/v2/en/indicator/SP.DYN.CDRT.IN?downloadformat=excel"
death_rate_para = "death_rate"

rural_pop = "https://api.worldbank.org/v2/en/indicator/SP.RUR.TOTL?downloadformat=excel"
rural_pop_para = "rural_pop"

age_dep_young = "https://api.worldbank.org/v2/en/indicator/SP.POP.DPND.YG?downloadformat=excel"
age_dep_young_para = "age_dep_young"

age_dep_old = "https://api.worldbank.org/v2/en/indicator/SP.POP.DPND.OL?downloadformat=excel"
age_dep_old_para = "age_dep_old"

fertilty_rate = "https://api.worldbank.org/v2/en/indicator/SP.DYN.TFRT.IN?downloadformat=excel"
fertilty_rate_para = "fertilty_rate"

inflation = "https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=excel"
inflation_para = "inflation"

gdp_growth = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.KD.ZG?downloadformat=excel"
gdp_growth_para = "gdp_growth"

gdp_percapita_growth = "https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.KD.ZG?downloadformat=excel"
gdp_percapita_growth_para = "gdp_percapita_growth"

foreign_inv = "https://api.worldbank.org/v2/en/indicator/BN.KLT.DINV.CD?downloadformat=excel"
foreign_inv_para = "foreign_inv"

net_migration = "https://api.worldbank.org/v2/en/indicator/SM.POP.NETM?downloadformat=excel"
net_migration_para = "net_migration"

agri_land = "https://api.worldbank.org/v2/en/indicator/AG.LND.AGRI.K2?downloadformat=excel"
agri_land_para = "agri_land"

oil_rents = "https://api.worldbank.org/v2/en/indicator/NY.GDP.PETR.RT.ZS?downloadformat=excel"
oil_rents_para = "oil_rents"

coal_rents = "https://api.worldbank.org/v2/en/indicator/NY.GDP.COAL.RT.ZS?downloadformat=excel"
coal_rents_para = "coal_rents"

edu_enroll = "https://api.worldbank.org/v2/en/indicator/SE.PRM.ENRR?downloadformat=excel"
edu_enroll_para = "edu_enroll"

electricty_csmp = "https://api.worldbank.org/v2/en/indicator/EG.USE.ELEC.KH.PC?downloadformat=excel"
electricty_csmp_para = "electricty_csmp"

In [69]:
links = [GDP, gdp_percapita, life_exp_female, life_exp_male,
         mortality_male, mortality_female, death_rate, rural_pop,age_dep_young,
         age_dep_old, fertilty_rate, inflation, gdp_growth, gdp_percapita_growth,
         foreign_inv, net_migration, agri_land, oil_rents, coal_rents, edu_enroll,
         electricty_csmp]

features = [gdp_para, gdp_percapita_para, life_exp_female_para,
            life_exp_male_para, mortality_male_para, mortality_female_para,
            death_rate_para, rural_pop_para ,age_dep_young_para, age_dep_old_para,
            fertilty_rate_para, inflation_para, gdp_growth_para,
            gdp_percapita_growth_para, foreign_inv_para, net_migration_para,
            agri_land_para, oil_rents_para, coal_rents_para, edu_enroll_para,
            electricty_csmp_para]

In [11]:
countries = pd.read_csv(COUNTRY_NAMES, usecols=["name", "alpha-3"])
print(countries.head())
countries['name'] = countries['name'].apply(str.lower).apply(str.strip)
print(countries.head())

             name alpha-3
0     Afghanistan     AFG
1   Åland Islands     ALA
2         Albania     ALB
3         Algeria     DZA
4  American Samoa     ASM
             name alpha-3
0     afghanistan     AFG
1   åland islands     ALA
2         albania     ALB
3         algeria     DZA
4  american samoa     ASM


In [112]:
#TODO: Drop time code

warnings.simplefilter(action='ignore', category=FutureWarning)

def simplify_country(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"Country Name": "name", "Country Code": "code"})
    df["name"] = df["name"].apply(str.lower).apply(str.strip)
    return df

def prepare_df(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    new_df = simplify_country(df)
    
    first_mean = new_df.drop(columns=["name", "code"]).iloc[:, 0].mean()
    last_mean = new_df.iloc[:, -1].mean()
    
    null_rows = new_df[new_df.isna().any(axis=1)]
    new_df.dropna(inplace=True)
    if (null_rows.empty):
        print(col_name)
        return new_df
    
    name_code = null_rows.loc[:, ['name', 'code']]
    null_rows = null_rows.drop(columns=['name', 'code']).transpose()
    null_rows.index = pd.Index([pd.to_datetime(i) for i in df.columns[2:]])
    
    for d in null_rows.columns:
        null_rows[d] = null_rows[d].interpolate(method='time')
        if null_rows[d].isna().any():
            null_rows[d] = null_rows[d].interpolate(option='spline')
        else: continue
        if null_rows[d].isna().any():
            if np.isnan(null_rows[d].iloc[0]):
                null_rows[d].iloc[0] = first_mean
            if np.isnan(null_rows[d].iloc[-1]):
                null_rows[d].iloc[-1] = last_mean
            null_rows[d] = null_rows[d].interpolate(method='linear')
        else:
            continue
        if null_rows[d].isna().any():
            null_rows[d] = null_rows[d].fillna(method='bfill')  
        else: continue
                
    null_rows = null_rows.transpose()
    null_rows.columns = [str(i.year) for i in null_rows.columns]
    null_rows['code'] = name_code.loc[null_rows.index, 'code']
    null_rows['name'] = name_code.loc[null_rows.index, 'name']
    
    assert (set(new_df.columns) == set(null_rows.columns))
    
    return new_df
        
    new_df = pd.concat([new_df, null_rows], ignore_index=True)
    
    new_df = pd.melt(new_df, id_vars=['name', 'code'], value_vars=[i for i in new_df.columns if i not in ['code, name']])
    new_df = new_df.rename(columns={"variable": "year", "value": col_name})
    
    return new_df.dropna(axis=0)
    
NOT_COLS = ["Indicator Name","Indicator Code"]
def get_df(link, feat_name, save=False, path=None):
    global NOT_COLS
    try:
        data = pd.read_excel(link, usecols=lambda x : x not in NOT_COLS, skiprows=3)
    except UnicodeDecodeError:
        data = pd.read_csv(link, usecols=lambda x : x not in NOT_COLS, skiprows=3)
    assert not save or save is not None

    new_data = prepare_df(data, feat_name)
    if save:
        new_data.to_excel(os.path.join(os.getcwd(), path))
    return new_data

In [113]:
dfs = [get_df(l, fn, save=True, path=os.path.join('generated_data', f"{fn}.xlsx")) for l, fn in zip(links, features)]

In [107]:
final_data = get_df(pop_growth, pop_growth_para).copy()
growth_data = final_data.copy()

print(growth_data.head(5))

def common_indices(df):
    global growth_data
    d_temp = df.copy()
    s1 = (set(list(d_temp.set_index(['code', 'year']).index)))
    s2 = (set(list(growth_data.set_index(['code', 'year']).index)))    
    return len(s1.intersection(s2))

for d in dfs:
    print(common_indices(d))

                          name code  year  pop_growth
0                        aruba  ABW  1960    2.179059
1  africa eastern and southern  AFE  1960    2.660180
2                  afghanistan  AFG  1960    1.925952
3   africa western and central  AFW  1960    2.115789
4                       angola  AGO  1960    1.558355
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16768
16768
16960
16960
16512
15872
15872
16448
16960


In [108]:
# in order to maximize the number of samples in the final generated dataset, sort the dataframes according
# to the number of common rows with the population's growth data.
dfs_sorted = sorted(dfs, key=lambda x:common_indices(x), reverse=True)
for d in dfs_sorted:
    print (common_indices(d))

16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16960
16768
16768
16512
16448
15872
15872


In [111]:
for df, l in zip(dfs_sorted, features):
    print(f"feat {l}")
    final_data = pd.merge(final_data, df.drop('name', axis=1), on=['code', 'year'], how='inner')
    print(f"the shape after merging {final_data.shape}")

feat gdp
the shape after merging (16768, 21)
feat gdp_percapita
the shape after merging (16768, 22)
feat life_exp_female
the shape after merging (16768, 23)
feat life_exp_male
the shape after merging (16768, 24)
feat mortality_male
the shape after merging (16768, 25)
feat mortality_female
the shape after merging (16768, 26)
feat death_rate
the shape after merging (16768, 27)
feat rural_pop
the shape after merging (16768, 28)
feat age_dep_young
the shape after merging (16768, 29)
feat age_dep_old
the shape after merging (16768, 30)
feat fertilty_rate
the shape after merging (16768, 31)
feat inflation
the shape after merging (16768, 32)
feat gdp_growth
the shape after merging (16768, 33)
feat gdp_percapita_growth
the shape after merging (16768, 34)
feat foreign_inv
the shape after merging (16768, 35)
feat net_migration
the shape after merging (16768, 36)
feat agri_land
the shape after merging (16768, 37)
feat oil_rents
the shape after merging (16384, 38)
feat coal_rents
the shape after m

In [110]:
final_data.to_excel(os.path.join('generated_data', f'final_dataset.xlsx'))